#### Cleaning each file and merging them.

In [11]:
import pandas as pd
import numpy as np
import re

# Load the CSV files
df1 = pd.read_csv('../data/raw/asrs_merged_file.csv', skiprows=1)  # Skip the first row with column names
df2 = pd.read_csv('../data/raw/ntsb_crash_reports.csv')

# Function to clean a DataFrame
def clean_dataframe(df):
    # 1. Check for missing values
    print("Missing values in each column:")
    print(df.isnull().sum())

    # 2. Handle missing values (drop or fill)
    df.fillna(method='ffill', inplace=True)  # Forward fill for missing values

    # 3. Data type consistency
    # Convert date columns to datetime if applicable
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # 4. Data cleansing: Remove whitespace and special characters
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.strip()  # Remove leading/trailing whitespace
        df[col] = df[col].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)  # Remove special characters

    # 5. Normalize column names (optional)
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]  # Convert to lowercase and replace spaces with underscores

    return df

# Clean both DataFrames
cleaned_df1 = clean_dataframe(df1)
cleaned_df2 = clean_dataframe(df2)

# Save the cleaned DataFrames to new CSV files (optional)
cleaned_df1.to_csv('../data/processed/asrs_data_cleaned.csv', index=False)
cleaned_df2.to_csv('../data/processed/ntsb_data_cleaned.csv', index=False)

print("Data cleaning complete. Cleaned files saved.")

C:\Users\Mary\AppData\Local\Temp\ipykernel_16052\2465256420.py:7: DtypeWarning: Columns (7,16,17,23,27,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('../data/raw/ntsb_crash_reports.csv')


Missing values in each column:
ACN                      0
Date                     0
Local Time Of Day      429
Locale Reference       327
State Reference        181
                     ...  
Callback             11562
Narrative.1          10767
Callback.1           11783
Synopsis                 2
Unnamed: 125         11812
Length: 126, dtype: int64


C:\Users\Mary\AppData\Local\Temp\ipykernel_16052\2465256420.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)  # Forward fill for missing values
C:\Users\Mary\AppData\Local\Temp\ipykernel_16052\2465256420.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


Missing values in each column:
NtsbNo                      0
EventType                  30
Mkey                        0
EventDate                   7
City                       61
State                    9299
Country                   508
ReportNo               176261
N                         103
HasSafetyRec                0
ReportType               7078
OriginalPublishDate     20243
HighestInjuryLevel     104717
FatalInjuryCount            0
SeriousInjuryCount          0
MinorInjuryCount            0
ProbableCause          114958
EventID                 91239
Latitude                    0
Longitude                   0
Make                       63
Model                     153
AirCraftCategory          607
AirportID              125163
AirportName             63805
AmateurBuilt                0
NumberOfEngines          6373
Scheduled              163590
PurposeOfFlight         14055
FAR                      6344
AirCraftDamage           4366
WeatherCondition         5339
Operator 

C:\Users\Mary\AppData\Local\Temp\ipykernel_16052\2465256420.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)  # Forward fill for missing values


Data cleaning complete. Cleaned files saved.


In [ ]:
import pandas as pd

# Suppress warnings
import warnings
warnings.simplefilter("ignore", category=pd.errors.DtypeWarning)
warnings.simplefilter("ignore", category=UserWarning)

# Step 1: Load datasets (with low_memory=False to suppress DtypeWarning)
print("Loading ASRS and NTSB datasets...")
asrs = pd.read_csv('../data/processed/asrs_data_cleaned.csv')  # Replace with the path to ASRS data
ntsb = pd.read_csv('../data/processed/ntsb_data_cleaned.csv', low_memory=False)  # Replace with the path to NTSB data

# Step 2: Selecting relevant columns from both datasets
print("Selecting relevant columns from ASRS and NTSB datasets...")
asrs_cols = [
    'date', 'local_time_of_day', 'state_reference', 'latitude_/_longitude_(uas)',
    'flight_conditions', 'aircraft_operator', 'make_model_name', 'flight_phase',
    'narrative', 'synopsis', 'primary_problem'
]
ntsb_cols = [
    'eventdate', 'state', 'latitude', 'longitude', 'make', 'model',
    'highestinjurylevel', 'fatalinjurycount', 'seriousinjurycount',
    'minorinjurycount', 'weathercondition', 'aircraftdamage'
]

# Filter datasets to include only relevant columns
asrs_filtered = asrs[asrs_cols]
ntsb_filtered = ntsb[ntsb_cols].copy()  # Create a copy to avoid view issues

# Step 3: Show number of rows in each dataset
print(f"Rows in ASRS dataset: {len(asrs_filtered)}")
print(f"Rows in NTSB dataset: {len(ntsb_filtered)}")

# Step 4: Convert date columns and format them to remove time component
print("Converting date columns and formatting them to remove time components...")
asrs_filtered.loc[:, 'date'] = pd.to_datetime(asrs_filtered['date'], errors='coerce').dt.strftime('%m/%d/%Y')
ntsb_filtered.loc[:, 'eventdate'] = pd.to_datetime(ntsb_filtered['eventdate'], errors='coerce').dt.strftime('%m/%d/%Y')

# Step 5: Create state abbreviation mapping for merging consistency
print("Creating state abbreviation mapping for merging consistency...")
state_mapping = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Map NTSB states to abbreviations
ntsb_filtered.loc[:, 'state_abbr'] = ntsb_filtered['state'].map(state_mapping)

# Step 6: Merge datasets on date, state, and aircraft make/model
print("Merging the ASRS and NTSB datasets based on date, state, and aircraft make/model...")
merged = pd.merge(
    asrs_filtered,
    ntsb_filtered,
    left_on=['date', 'state_reference', 'make_model_name'],
    right_on=['eventdate', 'state_abbr', 'make'],
    how='outer',  # Perform an outer join to include all records from both datasets
    suffixes=('_asrs', '_ntsb')
)

# Step 7: Show number of rows after merge
print(f"Rows in merged dataset: {len(merged)}")

# Step 8: Calculate how many rows were dropped during the merge (if any)
rows_dropped = len(asrs_filtered) + len(ntsb_filtered) - len(merged)
print(f"Rows dropped during merge: {rows_dropped}")

# Step 9: Save the merged dataset to a CSV file
print("Saving the merged dataset to a CSV file...")
merged.to_csv('../data/processed/merged_asrs_ntsb.csv', index=False)

Loading ASRS and NTSB datasets...
Selecting relevant columns from ASRS and NTSB datasets...
Rows in ASRS dataset: 11812
Rows in NTSB dataset: 176620
Converting date columns and formatting them to remove time components...
Creating state abbreviation mapping for merging consistency...
Merging the ASRS and NTSB datasets based on date, state, and aircraft make/model...
Rows in merged dataset: 188432
Rows dropped during merge: 0
Saving the merged dataset to a CSV file...
